In [3]:
import pandas as pd
import numpy as np
import sklearn
df = pd.read_excel("dataframe2.xlsx", sheet_name="Sheet1")
# %：困难模式/普通模式
# Double 有双个的字母：1（mummy，condo），没有双个的字母：0（china）
# Vowel  有多个元音：1（condo，aorta），没有多个元音：0（third）
# cumDup有多少个同样字母：2（mummy），1（foggy），0（plain）
# cumVowel 有多少个元音：3，2，1，0
# percentage of hard mode：基本没用
# uncommonwords：直译为不寻常的单词，有多个双个字母（cacao），有三个一样的字母（mummy，fluff），有三个元音（aorta），没有元音（glyph）
# easywords：1try+2tries，
# easywordsband：分成4类
# hardwords：6tries+7tries
# hardwordsband：分成4类


In [4]:
df.drop(['Unnamed: 0','percentage of hard mode','%','easywords','hardwords','easywordsband','hardwordsband'], axis=1, inplace=True)
df = df.sort_values('Contest number', ascending=True)
df.drop('Contest number', axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)
df_label = df.iloc[:, 1:10].copy()
df_label
# Double 有双个的字母：1（mummy，condo），没有双个的字母：0（china）
# Vowel  有多个元音：1（condo，aorta），没有多个元音：0（third）
# cumDup有多少个同样字母：2（mummy），1（foggy），0（plain）
# cumVowel 有多少个元音：3，2，1，0
# uncommonwords：直译为不寻常的单词，有多个双个字母（cacao），有三个一样的字母（mummy，fluff），有三个元音（aorta），没有元音（glyph）

,Number of reported results,Number in hard mode,1 try,2 tries,3 tries,4 tries,5 tries,6 tries,7 or more tries (X)
0,80630,1362,1,3,23,39,24,9,1
1,101503,1763,1,5,23,31,24,14,2
2,91477,1913,1,3,13,27,30,22,4
3,107134,2242,1,4,16,30,30,17,2
4,153880,3017,1,9,35,34,16,5,1
...,...,...,...,...,...,...,...,...,...
354,20879,2012,0,2,17,35,29,14,3
355,20160,1937,0,3,21,40,25,9,1
356,20001,1919,0,2,16,38,30,12,2
357,21204,1973,0,4,21,38,26,9,1


In [5]:
df.drop(['Word','Number of  reported results','Number in hard mode','1 try','2 tries','3 tries','4 tries','5 tries','6 tries','7 or more tries (X)'], axis=1, inplace=True)
df['date'] = df.index
df.drop('date', inplace=True, axis=1)
df_feature = df.copy()
df_feature

,double,vowel,cumDup,cumVowel,uncommonwords
0,0,0,0,1,0
1,0,0,0,1,0
2,1,1,1,2,0
3,0,1,0,2,0
4,0,0,0,1,0
...,...,...,...,...,...
354,1,1,1,2,0
355,0,1,0,2,0
356,0,1,0,2,0
357,0,1,0,2,0


In [6]:
from sklearn.preprocessing import MinMaxScaler
scalar = MinMaxScaler(feature_range=(0,1))
df_feature = scalar.fit_transform(df_feature)
df_feature = pd.DataFrame(df_feature, columns=df.columns)

In [7]:
divide_point = 359
df_train = df_feature.iloc[:divide_point, :].copy()
df_test = df_feature.iloc[divide_point:, :].copy()
df_train_label = df_label.iloc[:divide_point, :].copy()
df_test_label = df_label.iloc[divide_point:, :].copy()

In [8]:
import numpy as np
data_features = df_train.values
data_labels = df_train_label.values

time_step = 100

data_seq_feature = []
for i in range(len(data_features) - time_step + 1):
    data_seq_feature.append(data_features[i: i + time_step])
data_seq_feature = np.array(data_seq_feature)
data_seq_label = data_labels[time_step - 1 : ]
data_seq_feature.shape

(260, 100, 5)

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

input_dim = 5
output_dim = 9

model = Sequential()
model.add(LSTM(16, input_shape=(time_step, input_dim)))
model.add(Dropout(0.2))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(output_dim, activation='softmax'))

optimizer = tf.keras.optimizers.Adam(0.05)

model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])
model.fit(data_seq_feature, data_seq_label, epochs=5, batch_size=4)

    

ModuleNotFoundError: No module named 'tensorflow'